In [3]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Carica il modello pre-addestrato EfficientNetB0 senza i fully connected layers
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Congela i pesi del modello pre-addestrato
base_model.trainable = False

# Aggiungi strati personalizzati per la classificazione
x = base_model.output
x = GlobalAveragePooling2D()(x)  # GlobalAveragePooling sostituisce i fully connected layers
x = Dense(128, activation='relu')(x)  # Strato denso intermedio
predictions = Dense(10, activation='softmax')(x)  # 10 classi per 10 generi musicali

# Crea il modello finale
model = Model(inputs=base_model.input, outputs=predictions)

# Compila il modello
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Carica i dati degli spettrogrammi (usa ImageDataGenerator per caricare i dati dalle cartelle)
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # Ridimensiona i valori dei pixel

train_generator = train_datagen.flow_from_directory(
    'Data/images_original_3s/',  # Cartella con i tuoi spettrogrammi
    target_size=(224, 224),  # Dimensione di input richiesta da EfficientNetB0
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'Data/images_original_3s/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Addestra il modello
history = model.fit(train_generator, validation_data=validation_generator, epochs=10)

# ---- Parte 1: Grafici dell'addestramento ----
def plot_training_history(history):
    # Estrai i dati dalla cronologia
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc) + 1)

    # Plot della precisione
    plt.figure(figsize=(14, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'b', label='Training accuracy')
    plt.plot(epochs, val_acc, 'r', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot della perdita
    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

# Chiama la funzione per tracciare i grafici
plot_training_history(history)

# ---- Parte 2: Stampa dei valori dell'ultima epoca ----
def print_last_epoch_metrics(history):
    last_epoch = len(history.history['accuracy']) - 1
    print(f"Valori dell'ultima epoca (epoca {last_epoch + 1}):")
    print(f"Accuracy di training: {history.history['accuracy'][last_epoch]}")
    print(f"Accuracy di validation: {history.history['val_accuracy'][last_epoch]}")
    print(f"Loss di training: {history.history['loss'][last_epoch]}")
    print(f"Loss di validation: {history.history['val_loss'][last_epoch]}")

# Stampa i valori dell'ultima epoca
print_last_epoch_metrics(history)


ModuleNotFoundError: No module named 'tensorflow'